In [ ]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Data


In [ ]:
def Ch(Re):
  if Re < 10:
    return 0.718
  elif Re < 100:
    return 0.4
  else:
    return 0.3

def n(Re):
  if Re < 10:
    return 0.349
  elif Re < 100:
    return 0.598
  else:
    return 0.663

def Kp(Re):
  if Re < 15:
    return 47
  elif Re < 300:
    return 18.290
  else:
    return 1.441

def m(Re):
  if Re < 15:
    return 1
  elif Re < 300:
    return 0.652
  else:
    return 0.206

# Design Spesifications

In [ ]:
port_size = np.linspace(0.001,0.39,100) # in m
number_of_plates = np.linspace(3
                               ,700,698)
plate_thickness = np.linspace(0.0005,0.0012,71) # in m
plate_spacing = np.linspace(0.0015,0.005,351) # in m DONT NEED, WILL BE CALCULATED LATER 
MAX_PRESSURE_DROP = 25 # psi

In [ ]:
df = pd.read_csv("DESIGN PARAMETERS COMBINED.csv", index_col = 0)


In [ ]:
df

,PORT_SIZE,THICKNESS,PLATE_SPACING
0,0.001000,0.0005,0.0015
1,0.004929,0.0005,0.0015
2,0.008859,0.0005,0.0015
3,0.012788,0.0005,0.0015
4,0.016717,0.0005,0.0015
...,...,...,...
2492095,0.374283,0.0012,0.0050
2492096,0.378212,0.0012,0.0050
2492097,0.382141,0.0012,0.0050
2492098,0.386071,0.0012,0.0050


In [ ]:
effective_number_of_plates = number_of_plates - 2
NUMBER_OF_PASSES = 1 # Np
number_of_channel_per_pass = (number_of_plates - 1)/(2*NUMBER_OF_PASSES)

# Design Goals

In [ ]:
# Hot Fluid
Thi = 100 # degree celcius
Tho = 45 # degree celcius
# Cold Fluid
Tci = 25 # degree celcius
Tco = 35 # degree celcius
LOG_MEAN_TEMPERATURE_DIFF = ((Thi-Tco) - (Tho-Tci))/math.log((Thi-Tco)/(Tho-Tci))
print("The Log Mean Temperature Difference is :", LOG_MEAN_TEMPERATURE_DIFF)

The Log Mean Temperature Difference is : 38.17911105427177


# Material Parameters

In [ ]:
CP_MIXTURE = 7774.85 # J/kg*K
CP_WATER = 4178 # J/kg*K

DENSITY_MIXTURE = 980.3987 # kg/m3
DENSITY_WATER = 995.65 # kg/m3

VISCOSITY_MIXTURE = 4.548*10**(-4) # Pa*s # Obtained from other file 
VISCOSITY_WATER = 8.034*10**(-4) # Pa*s

THERMAL_CONDUCTIVITY_MIXTURE = 0.5096 # W/mK 
THERMAL_CONDUCTIVITY_WATER = 0.6172 # W/mK

PRANDTL_NUMBER_MIXTURE = VISCOSITY_MIXTURE*CP_MIXTURE/THERMAL_CONDUCTIVITY_MIXTURE
PRANDTL_NUMBER_WATER = VISCOSITY_WATER*CP_WATER/THERMAL_CONDUCTIVITY_WATER


In [ ]:
PRANDTL_NUMBER_WATER

5.438440051847053

In [ ]:
PRANDTL_NUMBER_MIXTURE

6.938779003139718

# Flow Parameters


In [ ]:
mass_flow_rate_of_mixture = round(13000/3600,3) # kg/s
Q = round(mass_flow_rate_of_mixture*CP_MIXTURE*(Thi-Tho),3) # J/s
mass_flow_rate_of_water = round(Q/(CP_WATER*(Tco-Tci)),3) # kg/s
print(f"Mass Flow Rate of Acetic Acid Mixture is {mass_flow_rate_of_mixture} kg/s\nHeat Transfer is {Q} J/s\nMass Flow Rate of Cooling Water is {mass_flow_rate_of_water} kg/s")

Mass Flow Rate of Acetic Acid Mixture is 3.611 kg/s
Heat Transfer is 1544124.084 J/s
Mass Flow Rate of Cooling Water is 36.958 kg/s


# Additional assumed parameters and same calculated parameters

In [ ]:
# Thermal conductivy of the material 
THERMAL_CONDUCTIVITY_OF_MATERIAL = 20 # W/m^2K
vertical_port_distance = 0.6 # m lv ALSO EFFECTIVE FLOW LENGTH
effective_channel_width = 0.9 # m lw
chevron_angle = 45 # degrees
A1 = 0.545 # m^2
A1p = (vertical_port_distance - port_size) * effective_channel_width
enlargement_factor = A1/A1p

In [ ]:
ratio = 0.6

A1 = A1 * ratio**2
vertical_port_distance = vertical_port_distance * ratio
effective_channel_width = effective_channel_width * ratio

In [ ]:
df["A1"] = A1
df["A1p"] = (vertical_port_distance - df["PORT_SIZE"]) * effective_channel_width # change 
df["E_FAC"] = df["A1"]/df["A1p"]

In [ ]:
df

,PORT_SIZE,THICKNESS,PLATE_SPACING,A1,A1p,E_FAC
0,0.001000,0.0005,0.0015,0.1962,0.193860,1.012071
1,0.004929,0.0005,0.0015,0.1962,0.191738,1.023270
2,0.008859,0.0005,0.0015,0.1962,0.189616,1.034721
3,0.012788,0.0005,0.0015,0.1962,0.187495,1.046430
4,0.016717,0.0005,0.0015,0.1962,0.185373,1.058408
...,...,...,...,...,...,...
2492095,0.374283,0.0012,0.0050,0.1962,-0.007713,-25.438472
2492096,0.378212,0.0012,0.0050,0.1962,-0.009835,-19.950083
2492097,0.382141,0.0012,0.0050,0.1962,-0.011956,-16.409672
2492098,0.386071,0.0012,0.0050,0.1962,-0.014078,-13.936459


# Calculating Parameters for Each Case and Adding Them to DataFrame

Dh = Hydraulic Diameter
Ach = One Channel Flow Area

In [ ]:
# Calculating Dh
df["Dh"] = 2*df["PLATE_SPACING"]/df["E_FAC"]
df["Ach"] = df["PLATE_SPACING"] * effective_channel_width

In [ ]:
df

,PORT_SIZE,THICKNESS,PLATE_SPACING,A1,A1p,E_FAC,Dh,Ach
0,0.001000,0.0005,0.0015,0.1962,0.193860,1.012071,0.002964,0.00081
1,0.004929,0.0005,0.0015,0.1962,0.191738,1.023270,0.002932,0.00081
2,0.008859,0.0005,0.0015,0.1962,0.189616,1.034721,0.002899,0.00081
3,0.012788,0.0005,0.0015,0.1962,0.187495,1.046430,0.002867,0.00081
4,0.016717,0.0005,0.0015,0.1962,0.185373,1.058408,0.002834,0.00081
...,...,...,...,...,...,...,...,...
2492095,0.374283,0.0012,0.0050,0.1962,-0.007713,-25.438472,-0.000393,0.00270
2492096,0.378212,0.0012,0.0050,0.1962,-0.009835,-19.950083,-0.000501,0.00270
2492097,0.382141,0.0012,0.0050,0.1962,-0.011956,-16.409672,-0.000609,0.00270
2492098,0.386071,0.0012,0.0050,0.1962,-0.014078,-13.936459,-0.000718,0.00270


# Mass Flow Rate Per Channel

In [ ]:
initial_number_of_plates = 10

design_list = []
final = pd.DataFrame(columns = ['PORT_SIZE', 'THICKNESS', 'PLATE_SPACING', 'A1', 'A1p', 'E_FAC', 'Dh',
       'Ach', 'Gch', 'Reh', 'Chh', 'nh', 'Nuh', 'hh', 'Gcc', 'Rec', 'Chc',
       'Nuc', 'hc', 'U', 'Uf', 'Q', 'Qf', 'Kph', 'Kpc', 'mh', 'mc', 'fh', 'fc', 'deltaPch',
       'deltaPcc', 'Number_of_plates'])
i = 1
while initial_number_of_plates <= 210:
  print(initial_number_of_plates)
  effective_number_of_plates = initial_number_of_plates - 2 # Ne
  number_of_channel_per_pass = (initial_number_of_plates - 1)/(2*NUMBER_OF_PASSES) # Ncp
  
  # Hot Fluid Parameters
  hot_mass_flow_rate_per_channel = mass_flow_rate_of_mixture/number_of_channel_per_pass
  df["Gch"] = hot_mass_flow_rate_per_channel/df["Ach"]
  df["Reh"] = df["Gch"]*df["Dh"]/VISCOSITY_MIXTURE
  df["Chh"] = df["Reh"].apply(Ch)
  df["nh"] = df["Reh"].apply(n)
  df["Nuh"] = df["Chh"]*df["Reh"]**df["nh"]*PRANDTL_NUMBER_MIXTURE**(1/3)
  df["hh"] = df["Nuh"] * THERMAL_CONDUCTIVITY_MIXTURE/df["Dh"]
  
  # Cold Fluid Parameters
  cold_mass_flow_rate_per_channel = mass_flow_rate_of_water/number_of_channel_per_pass
  df["Gcc"] = cold_mass_flow_rate_per_channel/df["Ach"]
  df["Rec"] = df["Gcc"]*df["Dh"]/VISCOSITY_WATER
  df["Chc"] = df["Rec"].apply(Ch)
  df["nh"] = df["Reh"].apply(n)
  df["Nuc"] = df["Chc"]*df["Rec"]**df["nh"]*PRANDTL_NUMBER_WATER**(1/3)
  df["hc"] = df["Nuc"] * THERMAL_CONDUCTIVITY_WATER/df["Dh"]
  
  df["U"] = (1/df["hh"] + 1/df["hc"] + df["THICKNESS"]/THERMAL_CONDUCTIVITY_OF_MATERIAL)**(-1)
  df["Uf"] = (1/df["U"] + 0.0000034)**(-1)


  Ae = effective_number_of_plates * A1
  
  df["Q"] = df["U"] * Ae * LOG_MEAN_TEMPERATURE_DIFF  
  df["Qf"] = df["Uf"] * Ae * LOG_MEAN_TEMPERATURE_DIFF
  # Fouiling_Factor
  
  df["Kph"] = df["Reh"].apply(Kp)
  df["Kpc"] = df["Rec"].apply(Kp)
  df["mh"] = df["Reh"].apply(m)
  df["mc"] = df["Rec"].apply(m)


  df["fh"] = df["Kph"]/(df["Reh"]**df["mh"])
  df["fc"] = df["Kpc"]/(df["Rec"]**df["mc"])

  df["deltaPch"] = (4*df["fh"]*vertical_port_distance*initial_number_of_plates/df["Dh"]*df["Gch"]**2/(2*DENSITY_MIXTURE))*0.000145038
  df["deltaPcc"] = (4*df["fc"]*vertical_port_distance*initial_number_of_plates/df["Dh"]*df["Gcc"]**2/(2*DENSITY_WATER))*0.000145038
  df["Gph"] = (4 * mass_flow_rate_of_mixture/(math.pi*df["PORT_SIZE"]**2))
  df["Gpc"] = (4 * mass_flow_rate_of_water/(math.pi*df["PORT_SIZE"]**2))
  
  df["deltaPph"] = 1.4*NUMBER_OF_PASSES*df["Gph"]**2/(2*DENSITY_MIXTURE)*0.000145038
  df["deltaPpc"] = 1.4*NUMBER_OF_PASSES*df["Gpc"]**2/(2*DENSITY_WATER)*0.000145038
  df["deltaPth"] = df["deltaPch"] + df["deltaPph"]
  df["deltaPtc"] = df["deltaPcc"] + df["deltaPpc"]

  
  tobeappened = list(df[(min(df["Q"]) == df["Q"])].values[0])
  tobeappened.insert(0,initial_number_of_plates)
  design_list.append(tobeappened)


  #print(df[min(df["deltaPpc"]) == df["deltaPpc"]][["deltaPcc","deltaPpc"]])
  da = df[((df["deltaPch"] + df["deltaPph"]) < 25) & ((df["deltaPcc"] + df["deltaPpc"]) < 25) & (df["Qf"] > 1.55*10**6)]
  da["Number_of_plates"] = initial_number_of_plates
  try:
    da = da[da.Q == min(da.Q)]
    print(da)
    final = pd.concat([final,da])
  except:
    pass
  
  
  i += 1
  initial_number_of_plates += 100
  
final

10
110
210
         PORT_SIZE  THICKNESS  PLATE_SPACING      A1       A1p     E_FAC  \
1859719   0.075657    0.00102        0.00495  0.1962  0.153545  1.277798   

               Dh       Ach        Gch         Reh  ...        fc  deltaPch  \
1859719  0.007748  0.002673  12.927431  220.224148  ...  0.330265  0.261908   

          deltaPcc         Gph          Gpc  deltaPph  deltaPpc  deltaPth  \
1859719  16.436231  803.238212  8221.012973  0.066814  6.891659  0.328721   

         deltaPtc  Number_of_plates  
1859719  23.32789               210  

[1 rows x 38 columns]


<ipython-input-37-2b68bb861d7a>:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  da["Number_of_plates"] = initial_number_of_plates


,PORT_SIZE,THICKNESS,PLATE_SPACING,A1,A1p,E_FAC,Dh,Ach,Gch,Reh,...,fc,deltaPch,deltaPcc,Number_of_plates,Gph,Gpc,deltaPph,deltaPpc,deltaPth,deltaPtc
1859719,0.075657,0.00102,0.00495,0.1962,0.153545,1.277798,0.007748,0.002673,12.927431,220.224148,...,0.330265,0.261908,16.436231,210,803.238212,8221.012973,0.066814,6.891659,0.328721,23.32789


In [ ]:
final[final.Q == min(final.Q)].iloc[0]

PORT_SIZE                 0.075657
THICKNESS                  0.00102
PLATE_SPACING              0.00495
A1                          0.1962
A1p                       0.153545
E_FAC                     1.277798
Dh                        0.007748
Ach                       0.002673
Gch                      12.927431
Reh                     220.224148
Chh                            0.3
nh                           0.663
Nuh                      20.458246
hh                     1345.626893
Gcc                     132.310165
Rec                    1275.952598
Chc                            0.3
Nuc                       60.45715
hc                     4816.154736
U                       998.220598
Uf                      994.844147
Q                   1555301.810029
Qf                  1550041.048162
Kph                          18.29
Kpc                          1.441
mh                           0.652
mc                           0.206
fh                        0.542832
fc                  